### Pregunta 1

In [10]:
def search(matrix):
    rows = len(matrix)
    cols = len(matrix[0])

    start = (0, 0)
    end = (rows - 1, cols - 1)

    current = start
    path = [current]

    while current != end:
        x, y = current
        neighbors = []

        # Movimiento de abajo y derecha
        if x + 1 < rows:
            neighbors.append((x + 1, y))  
        if y + 1 < cols:
            neighbors.append((x, y + 1))

        # Minimizamos la diferencia absoluta entre la temperatura actual y la temperatura objetivo
        next_move = min(
            neighbors, key=lambda n: abs(matrix[n[0]][n[1]] - matrix[end[0]][end[1]])
        )
        current = next_move
        path.append(current)

    return path

In [11]:
matriz_temperaturas = [
    [-18, -19, -20],
    [-16, -17, -20],
    [-15, -15, -20],
    ]

path = search(matriz_temperaturas)
for i, (row, col) in enumerate(path):
    print(f"Paso {i+1}: Posicion ({row}, {col}) - Temperatura: {matriz_temperaturas[row][col]}°C")
print("Destino alcanzado")

Paso 1: Posicion (0, 0) - Temperatura: -18°C
Paso 2: Posicion (0, 1) - Temperatura: -19°C
Paso 3: Posicion (0, 2) - Temperatura: -20°C
Paso 4: Posicion (1, 2) - Temperatura: -20°C
Paso 5: Posicion (2, 2) - Temperatura: -20°C
Destino alcanzado


### Pregunta 2

In [12]:
class Graph:
    def __init__(self, adjac_lis, matriz_temperaturas, matriz_costos_energia):
        self.adjac_lis = adjac_lis
        self.matriz_temperaturas = matriz_temperaturas
        self.matriz_costos_energia = matriz_costos_energia
        self.rows = len(matriz_temperaturas)
        self.cols = len(matriz_temperaturas[0])

    def get_neighbors(self, v):
        return self.adjac_lis.get(v, [])

    def distancia_manhattan(self, pos1, pos2):
        x1, y1 = pos1
        x2, y2 = pos2
        return abs(x2 - x1) + abs(y2 - y1)

    def get_position(self, node):
        return (node // self.cols, node % self.cols)

    def h(self, n):
        # Funcion euristica h(x) 
        current_pos = self.get_position(n)
        goal_pos = (self.rows - 1, self.cols - 1)
        
        base_distance = self.distancia_manhattan(current_pos, goal_pos)
        
        # Penalida de diferencia de temperatura
        temp_diff = abs(self.matriz_temperaturas[current_pos[0]][current_pos[1]] - 
                       self.matriz_temperaturas[goal_pos[0]][goal_pos[1]])
        
        # Penalida de costos de energia
        energy_penalty = self.matriz_costos_energia[current_pos[0]][current_pos[1]]
        
        return base_distance + temp_diff + energy_penalty

    def a_star_algorithm(self, start, stop):
            
        open_lst = set([start])
        closed_lst = set()

        # g(x)
        g_scores = {}  
        g_scores[start] = 0
        
        temp_diffs = {}
        temp_diffs[start] = 0

        parents = {}
        parents[start] = start

        while open_lst:
            current = None

            # Encontrar el nodo con el menor valor de f(x) = g(x) + h(x)
            for node in open_lst:
                if current is None or (g_scores[node] + self.h(node) < 
                                     g_scores[current] + self.h(current)):
                    current = node

            if current is None:
                print("Camino no encontrado")
                return None

            if current == stop:
                path = []
                while parents[current] != current:
                    path.append(current)
                    current = parents[current]
                path.append(start)
                path.reverse()
                print("\nCamino encontrado: {}".format(path))
                return path

            for neighbor, weight in self.get_neighbors(current):
                if neighbor >= self.rows * self.cols:
                    continue
                    
                current_pos = self.get_position(current)
                neighbor_pos = self.get_position(neighbor)
                
                # Calculando g(x)
                temp_diff = abs(self.matriz_temperaturas[current_pos[0]][current_pos[1]] - 
                              self.matriz_temperaturas[neighbor_pos[0]][neighbor_pos[1]])
                energy_cost = self.matriz_costos_energia[neighbor_pos[0]][neighbor_pos[1]]
                
                total_weight = energy_cost + temp_diff

                if neighbor not in open_lst and neighbor not in closed_lst:
                    open_lst.add(neighbor)
                    parents[neighbor] = current
                    g_scores[neighbor] = g_scores[current] + total_weight
                    temp_diffs[neighbor] = temp_diffs[current] + temp_diff
                else:
                    if g_scores[neighbor] > g_scores[current] + total_weight:
                        g_scores[neighbor] = g_scores[current] + total_weight
                        temp_diffs[neighbor] = temp_diffs[current] + temp_diff
                        parents[neighbor] = current
                        if neighbor in closed_lst:
                            closed_lst.remove(neighbor)
                            open_lst.add(neighbor)

            open_lst.remove(current)
            closed_lst.add(current)

        print("Camino no encontrado")
        return None

In [13]:
adjac_lis = {
    0: [(1, 1), (3, 1)],
    1: [(0, 1), (2, 1), (4, 1)],
    2: [(1, 1), (5, 1)],
    3: [(0, 1), (4, 1), (6, 1)],
    4: [(1, 1), (3, 1), (5, 1), (7, 1)],
    5: [(2, 1), (4, 1), (8, 1)],
    6: [(3, 1), (7, 1)],
    7: [(4, 1), (6, 1), (8, 1)],
    8: [(5, 1), (7, 1)]
}

matriz_temperaturas = [
    [-18, -19, -20],
    [-16, -17, -20],
    [-15, -15, -20],
]

matriz_costos_energia = [
    [5, 6, 7],
    [4, 5, 7],
    [3, 3, 7],
]

graph = Graph(adjac_lis, matriz_temperaturas, matriz_costos_energia)

start_node = 0
goal_node = 8

In [14]:
print(f"Empezando en el nodo(zona) {start_node} al nodo(zona) {goal_node}")
print(f"Temperatura inicial: {matriz_temperaturas[0][0]}°C")
print(f"Costo de Energia inicial: {matriz_costos_energia[0][0]}")

path = graph.a_star_algorithm(start_node, goal_node)

if path:
    for node in path:
        pos = graph.get_position(node)
        temp = matriz_temperaturas[pos[0]][pos[1]]
        energy = matriz_costos_energia[pos[0]][pos[1]]
        print(f"Nodo(Zona) {node}: Posicion={pos}, Temperatura={temp}°C, Costo Energetico={energy}")


Empezando en el nodo(zona) 0 al nodo(zona) 8
Temperatura inicial: -18°C
Costo de Energia inicial: 5

Camino encontrado: [0, 3, 6, 7, 8]
Nodo(Zona) 0: Posicion=(0, 0), Temperatura=-18°C, Costo Energetico=5
Nodo(Zona) 3: Posicion=(1, 0), Temperatura=-16°C, Costo Energetico=4
Nodo(Zona) 6: Posicion=(2, 0), Temperatura=-15°C, Costo Energetico=3
Nodo(Zona) 7: Posicion=(2, 1), Temperatura=-15°C, Costo Energetico=3
Nodo(Zona) 8: Posicion=(2, 2), Temperatura=-20°C, Costo Energetico=7


### Pregunta 3

In [15]:
import random

In [16]:
productos = [" Helado Premium",
            " Carne Congelada", 
            " Vegetales Congelados",
            " Pescado",
            " Yogurt Helado",
            " Queso Crema",
            " Mantequilla",
            " Pletas de Hielo"]

weights = [2, 3.5, 2.5, 4, 1, 0.5, 0.8, 1.5]
costs = [9, 8, 6, 7, 5, 4, 3, 5]
max_weight = 10


In [17]:
def total_value(solution):
    total_weight = sum(w * s for w, s in zip(weights, solution))
    total_cost = sum(c * s for c, s in zip(costs, solution))
    
    if total_weight > max_weight:
        return 0
    
    return total_cost

def generate_solution():
    while True:
        solution = [random.randint(0, 1) for _ in range(len(weights))]
        if sum(w * s for w, s in zip(weights, solution)) <= max_weight: # No pasar los 10 kg
            return solution

def hill_climbing(max_iterations=100):
    solution = generate_solution()
    value = total_value(solution)
    moves = 0
    
    for i in range(max_iterations):
        neighbor = solution[:]
        idx = random.randint(0, len(weights) - 1)
        neighbor[idx] = 1 - neighbor[idx]
        neighbor_value = total_value(neighbor)
        
        if neighbor_value > value:
            solution = neighbor[:]
            value = neighbor_value
            moves += 1

    return solution, value, moves


In [18]:
best_solution, best_cost, total_moves = hill_climbing()
print("Combinacion Final (Mejor Solucion):", best_solution)
print("Valor Total:", best_cost)
print("Peso Total:", sum(w * s for w, s in zip(weights, best_solution)))
print("Numero de movimientos realizados:", total_moves)

print("\nProductos seleccionados:")
for i, selected in enumerate(best_solution):
    if selected:
        print(f"{productos[i]}")

Combinacion Final (Mejor Solucion): [0, 1, 0, 1, 0, 0, 1, 1]
Valor Total: 23
Peso Total: 9.8
Numero de movimientos realizados: 1

Productos seleccionados:
 Carne Congelada
 Pescado
 Mantequilla
 Pletas de Hielo


### Pregunta 4

In [19]:
import numpy as np
import random
import math

def calculate_total_value(solution, weights, costs, max_weight):
    total_weight = sum(weights[i] for i in range(len(solution)) if solution[i])
    if total_weight > max_weight:
        return -float('inf')
    return sum(costs[i] for i in range(len(solution)) if solution[i])

def get_neighbor(solution):
    neighbor = solution.copy()
    num_flips = random.randint(1, 2)
    for _ in range(num_flips):
        idx = random.randint(0, len(neighbor)-1)
        neighbor[idx] = not neighbor[idx]
    return neighbor

def simulated_annealing():
    # Temeperatura inicial y el factor de enfriamiento
    T = 1000
    cooling_rate = 0.95
    min_temp = 0.01
    
    # Solucion inicial
    current_solution = [random.choice([True, False]) for _ in range(len(productos))]
    current_value = calculate_total_value(current_solution, weights, costs, max_weight)
    best_solution = current_solution.copy()
    best_value = current_value
    
    iterations = 0
    
    print(f"Temperatura Inicial: {T}")
    
    while T > min_temp:
        iterations += 1

        neighbor = get_neighbor(current_solution)
        neighbor_value = calculate_total_value(neighbor, weights, costs, max_weight)
        
        delta = neighbor_value - current_value
        
        if delta > 0 or random.random() < math.exp(delta/T):
            current_solution = neighbor
            current_value = neighbor_value
            
            if current_value > best_value:
                best_solution = current_solution.copy()
                best_value = current_value
        
        T *= cooling_rate
        
        if iterations % 2 == 0:
            print(f"Temperatura: {T:.2f}, Valor Actual: {current_value}")
    
    # Calcular el peso final
    final_weight = sum(weights[i] for i in range(len(best_solution)) if best_solution[i])
    
    print("Productos Seleccionados:", [productos[i] for i in range(len(best_solution)) if best_solution[i]])
    print(f"Peso Total: {final_weight}")
    print(f"Valor Total: {best_value}")
    print(f"Iteraciones: {iterations}")
    
    return best_solution, best_value

best_solution, best_value = simulated_annealing()


Temperatura Inicial: 1000
Temperatura: 902.50, Valor Actual: 13
Temperatura: 814.51, Valor Actual: 28
Temperatura: 735.09, Valor Actual: 26
Temperatura: 663.42, Valor Actual: 33
Temperatura: 598.74, Valor Actual: 33
Temperatura: 540.36, Valor Actual: 26
Temperatura: 487.67, Valor Actual: 9
Temperatura: 440.13, Valor Actual: 21
Temperatura: 397.21, Valor Actual: 12
Temperatura: 358.49, Valor Actual: 17
Temperatura: 323.53, Valor Actual: 22
Temperatura: 291.99, Valor Actual: 29
Temperatura: 263.52, Valor Actual: 27
Temperatura: 237.83, Valor Actual: 17
Temperatura: 214.64, Valor Actual: 26
Temperatura: 193.71, Valor Actual: 26
Temperatura: 174.82, Valor Actual: 25
Temperatura: 157.78, Valor Actual: 19
Temperatura: 142.40, Valor Actual: 14
Temperatura: 128.51, Valor Actual: 12
Temperatura: 115.98, Valor Actual: 21
Temperatura: 104.67, Valor Actual: 25
Temperatura: 94.47, Valor Actual: 25
Temperatura: 85.26, Valor Actual: 15
Temperatura: 76.94, Valor Actual: 18
Temperatura: 69.44, Valor Ac